In [18]:
import numpy as np 
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import itertools
import cv2
import seaborn as sns
from PIL import Image



In [6]:
!pip install opencv-python


In [7]:
!pip install tensorflow


In [ ]:
import tensorflow as tf 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Loading dataset

In [ ]:
import os
import pandas as pd

dataset_dir = "Dataset"

data = []
labels = []

for fold in os.listdir(dataset_dir):
    fold_path = os.path.join(dataset_dir, fold)

    if os.path.isdir(fold_path):
        for file in os.listdir(fold_path):
            file_path = os.path.join(fold_path, file)
            data.append(file_path)
            labels.append(fold)

dataset = pd.DataFrame({
    "paths": data,
    "labels": labels
})

print(dataset.head())
print("Total images:", len(dataset))


                                              paths       labels
0  Dataset\Anthracnose\20211008_124249 (Custom).jpg  Anthracnose
1  Dataset\Anthracnose\20211008_124250 (Custom).jpg  Anthracnose
2  Dataset\Anthracnose\20211008_124252 (Custom).jpg  Anthracnose
3  Dataset\Anthracnose\20211008_124253 (Custom).jpg  Anthracnose
4  Dataset\Anthracnose\20211008_124256 (Custom).jpg  Anthracnose
Total images: 4000


In [14]:
dataset.head()

,paths,labels
0,Dataset\Anthracnose\20211008_124249 (Custom).jpg,Anthracnose
1,Dataset\Anthracnose\20211008_124250 (Custom).jpg,Anthracnose
2,Dataset\Anthracnose\20211008_124252 (Custom).jpg,Anthracnose
3,Dataset\Anthracnose\20211008_124253 (Custom).jpg,Anthracnose
4,Dataset\Anthracnose\20211008_124256 (Custom).jpg,Anthracnose


## Splitting the data

In [15]:
training_data, temp_validation_data = train_test_split(dataset, test_size=0.2, shuffle=True, random_state=42, stratify=dataset['labels'])
validation_data, test_data = train_test_split(temp_validation_data, test_size=0.05, shuffle=True, random_state=42, stratify=temp_validation_data['labels'])
print(f"Dimension of the training data: {training_data.shape[0]}")
print(f"Dimension of the validation data: {validation_data.shape[0]}")
print(f"Dimension of the test data: {test_data.shape[0]}")

Dimension of the training data: 3200
Dimension of the validation data: 760
Dimension of the test data: 40


## Data Prep

In [17]:
# Batch size
batch_size = 32
# Dataset image shape
img_shape = (224, 224)

# The original dataset was augmentated
# This function only returns the image as is
def return_image(image):
    return image

# Create a ImageDataGenerator
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=return_image)
validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=return_image)

# Reading the images
training_data_gen = train_gen.flow_from_dataframe(training_data,
                                              x_col = "paths",
                                              y_col="labels",
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=img_shape,
                                              batch_size=batch_size,
                                              color_mode='rgb'                                             
                                             )
validation_data_gen = train_gen.flow_from_dataframe(validation_data,
                                              x_col = "paths",
                                              y_col="labels",
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=img_shape,
                                              batch_size=batch_size,
                                              color_mode='rgb'                                             
                                             )



Found 3200 validated image filenames belonging to 8 classes.
Found 760 validated image filenames belonging to 8 classes.
